In [ ]:
!pip install textblob
!pip install tweepy

In [2]:
# Import Libraries
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import pandas as pd
import re
from textblob import TextBlob
import time
import tweepy

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zjw64\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Authentication
def get_authen():
    apiKey = "mi3IsHw1lzA9KlstGoEi4sQSu"
    apiSecret = "0obn2WECJqrZt5eoYJoOY5HbHCQfa5Xe6PFW58EG3AEqF4dxUM"
    accessToken = "1441571989357957122-eksbtpbuyFyQiMnIm9t3DOkILvzXLV"
    accessTokenSecret = "aLssavUn0tPln5czeoWaVcf0UYSRYAoenbyBIdf7TPvBE"
    auth = tweepy.OAuthHandler(apiKey, apiSecret)
    auth.set_access_token(accessToken, accessTokenSecret)
    # api = tweepy.API(auth)
    api =tweepy.API(auth)
    return api

In [4]:
# get input
def get_keyword():
    keyword = input("Please enter keyword to search: ")
    return keyword

def get_numOfTweet():
    numOfTweet = input ("Please enter the number of tweets to analyze: ")
    return numOfTweet

def test_numOfTweet(numOfTweet):
    # number of input tweets should be an integer from 0 to 2000
    if numOfTweet.isdigit() and int(numOfTweet) <= 2000 and int(numOfTweet) >= 0:
        return numOfTweet
    else: 
        return "input_num_wrong"

In [5]:
# search tweets
def get_twi_list(keyword, numOfTweet):
    api = get_authen()
    tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(int(numOfTweet))
    tweet_list = []
    for tweet in tweets:
        tweet_list.append(tweet.text)

    tweet_list = pd.DataFrame(tweet_list)
    # delete duplicates
    tweet_list.drop_duplicates(inplace = True)

    return tweet_list

In [6]:
# sentiment analyze
def sentiment_analyze(tweet_list):
    #Extracting text values
    tw_list = pd.DataFrame(tweet_list)
    if tw_list.empty:
        return tw_list
    tw_list["text"] = tw_list[0]

    #Cleaning Text (RT, Punctuation etc)
    tw_list = pd.DataFrame(tweet_list)
    tw_list["text"] = tw_list[0]
    remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
    rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
    tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
    tw_list["text"] = tw_list.text.str.lower()

    if tw_list.empty:
        return tw_list

    #Calculating Negative, Positive, Neutral and Compound values
    tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
    for index, row in tw_list['text'].iteritems():
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:
            tw_list.loc[index, 'sentiment'] = "negative"
        elif pos > neg:
            tw_list.loc[index, 'sentiment'] = "positive"
        else:
            tw_list.loc[index, 'sentiment'] = "neutral"
        tw_list.loc[index, 'neg'] = neg
        tw_list.loc[index, 'neu'] = neu
        tw_list.loc[index, 'pos'] = pos
        tw_list.loc[index, 'compound'] = comp

    #Creating new data frames for all sentiments (positive, negative and neutral)
    tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
    tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
    tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

    return tw_list

In [7]:
def count_values_in_column(data):
    #Function for count_values_in single columns
    feature = "sentiment"
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [8]:
def search_limit_alert(time_diff, search_num):
    # stop searching when search 15 times in 15 minutes
    if time_diff/60 < 15 and search_num >= 15:
        return True
    else:
        return False

In [ ]:
if __name__ == "__main__":
    # Instantiate an object
    start_time = time.time()
    search_num = 0

    while True:
        end_time = time.time()
        if search_limit_alert((end_time-start_time), search_num):
            print("-------------------------------------")
            print("You have no chance to search in 15 minites.")
            print()
            break

        print("-------------------------------------")
        keyword = get_keyword()

        while True:
            numOfTweet = get_numOfTweet()
            test = test_numOfTweet(numOfTweet)
            if test == "input_num_wrong":
                print("Please enter an integer from 0 to 2000.")
            else:
                break   

        tweet_list = get_twi_list(keyword, numOfTweet)
        print("Tweet total number: ",len(tweet_list))
        print()

        print("-------------------------------------")
        tw_list = sentiment_analyze(tweet_list)
        search_num = search_num + 1
        print("Sentiment Analysis: (10 samples)")
        if tw_list.empty:
            print("WARNING: 0 tweet found.")
        else:
            print(tw_list.head(11))
            print()
            print("-------------------------------------")
            print("Count Value for Sentiment Analysis: ")
            print(count_values_in_column(tw_list))
            print()

-------------------------------------
Please enter keyword to search: wechat status
Please enter the number of tweets to analyze: 500
Tweet total number:  187

-------------------------------------
Sentiment Analysis: (10 samples)
                                                    0  \
0   RT @jakaena1: Also a guy commented on one of h...   
1   WITHDRAW RM2000🎉\nJOIN NOW!!\nWelcome Bonus 30...   
2   RT @NaNcRaZy: กำลังนั่งอ่าน wechat ของ China H...   
4   برائے فروخت ۔ خریدنے کیلئے فالو کرو ۔ اور انبا...   
6   برائے فروخت ۔ خریدنے کیلئے فالو کرو ۔ اور انبا...   
9   Chinese man said some hurty words about traffi...   
10  RT @Buce2021: I suggest that if you have #FEG,...   
13  RT @Chikaofficials: GSC 1/8 สเกลฟิกเกอร์ #ปรมา...   
14  یاد اپ WeChat بخیر، فحش مینوشتیم میذاشتیم داخل...   
15  RT @WYB85_MeMories: 191030 #왕이보 #WangYibo\n▪Or...   
16  RT @bocheeeez: 公式がWeChatのフリート的な機能でマップ豆知識みたいなやつ...   

                                                 text  polarity  subjectivity  \
0  